Refeences:
- https://www.commandprompt.com/education/how-to-connect-to-postgresql-database-server-using-python/

# Initialization

In [1]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [36]:
# Connect to postgre database
try:
    con = psycopg2.connect(
    database="olist_database",
    user="postgres",
    password="789HibalK",
    host="localhost",
    port= '5432'
    )
    cursor_obj = con.cursor()
except:
  print("An exception occurred")
else:
   print("success")

success


In [37]:
# Check the data
query_example = pd.read_sql_query(
    '''
    SELECT 
        *
    FROM product_sales;
    ''',
    con)
example1 = pd.DataFrame(query_example)
example1.head()

,order_id,order_status,customer_id,order_purchase_timestamp,product_id,product_category_name,price,freight_value
0,00024acbcdf0a6daa1e931b038114c75,delivered,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,7634da152a4610f1595efa32f14722fc,perfumery,12.99,12.79
1,00061f2a7bc09da83e415a52dc8a4af1,delivered,c6fc061d86fab1e2b2eac259bac71a49,2018-03-24 22:16:10,d63c1011f49d98b976c352955b1c4bea,health_beauty,59.99,8.88
2,0008288aa423d2a3f00fcb17cd7d8719,delivered,2355af7c75e7c98b43a87b2a7f210dc5,2018-02-13 22:10:21,368c6c730842d78016ad823897a372db,garden_tools,49.90,13.37
3,0008288aa423d2a3f00fcb17cd7d8719,delivered,2355af7c75e7c98b43a87b2a7f210dc5,2018-02-13 22:10:21,368c6c730842d78016ad823897a372db,garden_tools,49.90,13.37
4,0009792311464db532ff765bf7b182ae,delivered,2a30c97668e81df7c17a8b14447aeeba,2018-08-14 20:43:09,8cab8abac59158715e0d70a36c807415,sports_leisure,99.90,27.65


In [38]:
example1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  112650 non-null  object        
 1   order_status              112650 non-null  object        
 2   customer_id               112650 non-null  object        
 3   order_purchase_timestamp  112650 non-null  datetime64[ns]
 4   product_id                112650 non-null  object        
 5   product_category_name     111047 non-null  object        
 6   price                     112650 non-null  float64       
 7   freight_value             112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 6.9+ MB


# Product Sales Analysis

## Product Order
The most ordered product category (Top 3 Product product category) in the last 6 months


In [46]:
# Product Order
query_prod_ord = pd.read_sql_query(
    '''
    WITH temp_tab AS (
        SELECT 
            product_id,
            product_category_name,
            EXTRACT('YEAR' FROM order_purchase_timestamp) AS year,
            EXTRACT('MONTH' FROM order_purchase_timestamp) AS month,
            COUNT(order_id) AS total_order
        FROM product_sales
        GROUP BY 1, 2, 3, 4
        ORDER BY year DESC, month DESC
    )
    SELECT 
        product_category_name,
        SUM(total_order) AS total_order
    FROM temp_tab
    WHERE year = 2018 and month > 4
    GROUP BY 1
    ORDER BY total_order DESC
    LIMIT 3;
    ''',
    con)
result1 = pd.DataFrame(query_prod_ord)
result1

,product_category_name,total_order
0,health_beauty,3288.0
1,bed_bath_table,2802.0
2,housewares,2409.0


## Product Sales
Total Sales and average monthly sales for each product category
(In 2018)

In [52]:
# Product Order
query_prod_sales = pd.read_sql_query(
    '''
    SELECT 
        temp_table.product_category_name,
        SUM(temp_table.total_sales) AS total_sales,
        AVG(temp_table.total_sales) AS average_monthly_sales
    FROM (
        SELECT 
            product_category_name,
            EXTRACT('YEAR' FROM order_purchase_timestamp) AS year,
            EXTRACT('MONTH' FROM order_purchase_timestamp) AS month,
            SUM(price) AS total_sales
        FROM product_sales
        GROUP BY 1, 2, 3
    ) AS temp_table
    WHERE year = 2018
    GROUP BY 1
    ORDER BY total_sales DESC
    LIMIT 10;
    ''',
    con)
result2 = pd.DataFrame(query_prod_sales)
result2

,product_category_name,total_sales,average_monthly_sales
0,health_beauty,772238.15,96529.76875
1,watches_gifts,708850.94,88606.36750
2,bed_bath_table,538069.26,67258.65750
3,sports_leisure,532566.49,66570.81125
4,computers_accessories,505476.31,63184.53875
5,housewares,399888.10,49986.01250
6,furniture_decor,386668.59,48333.57375
7,auto,347631.15,43453.89375
8,baby,256800.70,32100.08750
9,cool_stuff,240559.20,30069.90000


In [54]:
con.close()